In [ ]:
pip install pysoundfile

In [ ]:
import soundfile as sf

In [ ]:
data, samplerate = sf.read('sax/118__[sax][nod][jaz_blu]1702__3.wav')
sf.write('new_file.ogg', data, samplerate)

In [ ]:
pip install python_speech_features

In [ ]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import pandas as pd
import numpy as np
from statistics import stdev

In [ ]:
(rate,sig) = wav.read("sax/118__[sax][nod][jaz_blu]1702__3.wav")
mfcc_feat = mfcc(sig,rate, nfft = 5000)
fbank_feat = logfbank(sig,rate, nfft = 5000)

In [ ]:
fbank_feat.shape

In [ ]:
(rate,sig) = wav.read("vio/134__[vio][nod][cla]2156__3.wav")
mfcc_feat = mfcc(sig,rate, nfft = 1350)
fbank_feat = logfbank(sig,rate, nfft = 1350)

In [ ]:
fbank_feat[1:6,]

In [ ]:
pip install librosa

In [ ]:
import librosa
audio_path = 'vio/134__[vio][nod][cla]2156__3.wav'
x , sr = librosa.load(audio_path, sr =44100)

In [ ]:
import IPython.display as ipd
ipd.Audio(audio_path)

In [ ]:
mfccs = librosa.feature.mfcc(x, sr=sr)
mfccs.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

In [ ]:
#display Spectrogram
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
#If to pring log of frequencies  
#librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

In [ ]:
x, sr = librosa.load(audio_path, sr =None)
#Plot the signal:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

In [ ]:
y, sr = librosa.load(librosa.util.example_audio_file())

In [ ]:
mfccs = librosa.feature.mfcc(x, sr=sr)
mfccs

In [ ]:
mfccs.mean(axis=1)

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
import statistics

# Preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
header = 'filename rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [ ]:
Instruments = 'flu pia tru org gac voi'.split()
for i in Instruments:
    for filename in os.listdir(f'C:/Users/ankit/Desktop/CDS Project/{i}'):
        songname = f'C:/Users/ankit/Desktop/CDS Project/{i}/{filename}'
        y, sr = librosa.load(songname, sr =44100)
        rms = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {i}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
# Dropping unneccesary columns
df = df.drop(['filename'],axis=1)

In [ ]:
df1 = shuffle(df)

In [ ]:
df1.shape

In [ ]:
instru_list = df1.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(instru_list)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(df1.iloc[:, :-1], dtype = float))

In [ ]:
X_train.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(40 , activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=1)
test_loss, test_acc = model.evaluate(X_test,y_test)
pred = model.predict(X_test)
predicted_labels = np.argmax(pred, axis=1)
confusion_matrix(y_test, predicted_labels)

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

In [ ]:
print(history.history.keys())

In [ ]:
test_acc

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred

In [ ]:
confusion_matrix(y_test, np.argmax(pred, axis=1))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt  
df_cm = pd.DataFrame(confusion_matrix(y_test, predicted_labels),index=["flu", "gac", "org", "pia", "tru", "voi"], columns=["flu", "gac", "org", "pia", "tru", "voi"])
#plt.figure(figsize = (10,7))
sn.set(font_scale=1.0)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 18},fmt='g')# font size

plt.show()

#SVM

In [ ]:
svclassifier = SVC(kernel='rbf', C = 10.0, gamma=0.1, degree = 3)
svclassifier.fit(X_train, y_train)
predicted_labels = svclassifier.predict(X_test)
print("Recall: ", recall_score(y_test, predicted_labels,average=None))
print("Precision: ", precision_score(y_test, predicted_labels,average=None))
print("F1-Score: ", f1_score(y_test, predicted_labels, average=None))
print("Accuracy: %.2f  ," % accuracy_score(y_test, predicted_labels,normalize=True), accuracy_score(y_test, predicted_labels,normalize=False) )
print("Number of samples:",y_test.shape[0])
confusion_matrix(y_test, predicted_labels)

In [ ]:
df_cm = pd.DataFrame(confusion_matrix(y_test, predicted_labels),index=["flu", "pia", "tru", "org", "gac", "voi"], columns=["flu", "pia", "tru", "org", "gac", "voi"])
#plt.figure(figsize = (10,7))
sn.set(font_scale=1.0)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 18},fmt='g')# font size

plt.show()

#Cross-Validation

In [ ]:
df1.iloc[:, :-1] = scaler.fit_transform(np.array(df1.iloc[:, :-1], dtype = float))

In [ ]:
df1.iloc[:, -1] = encoder.fit_transform(df1.iloc[:, -1])

In [ ]:
df1

#Cross-Validation

In [ ]:
scores = cross_val_score(svclassifier, X, y, cv=10, scoring='accuracy')

In [ ]:
statistics.mean(scores)

#RandomForest

In [ ]:
RFClassifier = RandomForestClassifier(n_estimators=500, random_state=1993, max_depth=18)
RFClassifier.fit(X_train, y_train)
predicted_labels = RFClassifier.predict(X_test)
print("Recall: ", recall_score(y_test, predicted_labels,average=None))
print("Precision: ", precision_score(y_test, predicted_labels,average=None))
print("F1-Score: ", f1_score(y_test, predicted_labels, average=None))
print("Accuracy: %.2f  ," % accuracy_score(y_test, predicted_labels,normalize=True), accuracy_score(y_test, predicted_labels,normalize=False) )
print("Number of samples:",y_test.shape[0])
confusion_matrix(y_test, predicted_labels)

In [ ]:
RFClassifier = RandomForestClassifier(n_estimators=500, random_state=1993, max_depth=18)

In [ ]:
RFClassifier.fit(X_train, y_train)

In [ ]:
predicted_labels = RFClassifier.predict(X_test)

In [ ]:
print("Recall: ", recall_score(y_test, predicted_labels,average=None))
print("Precision: ", precision_score(y_test, predicted_labels,average=None))
print("F1-Score: ", f1_score(y_test, predicted_labels, average=None))
print("Accuracy: %.2f  ," % accuracy_score(y_test, predicted_labels,normalize=True), accuracy_score(y_test, predicted_labels,normalize=False) )

print("Number of samples:",y_test.shape[0])

In [ ]:
confusion_matrix(y_test, predicted_labels)

#xgboost

In [ ]:
XGBvlassifier = XGBClassifier(n_estimators=300, max_depth= 70, learning_rate=1)

In [ ]:
XGBvlassifier.fit(X_train, y_train)

In [ ]:
predicted_labels = XGBvlassifier.predict(X_test)

In [ ]:
print("Recall: ", recall_score(y_test, predicted_labels,average=None))
print("Precision: ", precision_score(y_test, predicted_labels,average=None))
print("F1-Score: ", f1_score(y_test, predicted_labels, average=None))
print("Accuracy: %.2f  ," % accuracy_score(y_test, predicted_labels,normalize=True), accuracy_score(y_test, predicted_labels,normalize=False) )

print("Number of samples:",y_test.shape[0])

In [ ]:
confusion_matrix(y_test, predicted_labels)